In [1]:
from glob import glob

import os
import pandas as pd
import numpy as np
import json
from pathlib import Path
import torch
import pickle

In [2]:
order = [
    "DeepFool", "BIM", "CW", "Low_CW", 
    "FAB", "FGSM", "PGD", "Low_PGD1", "Low_PGD2", 
    "AutoAttack", "Square"
]

In [53]:
dataname = 'SVHN'
modelname = 'resnet34'

# Adversarial Attack

In [54]:
del_keys = ['clean acc','noise acc','clean dwt acc']

attack_results = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/results.json')
adv_df = pd.DataFrame()

for p in attack_results:
    r = json.load(open(p,'r'))
    
    for k in del_keys:
        del r[k]
    
    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    adv_df = pd.concat([adv_df, r], axis=1)
    
adv_df = ((adv_df.T.loc[order]) * 100).round(2)

adv_df.columns = ['Adv Acc(%)','Adv Acc(%) DWT']

## Successed Images

In [55]:
succ_list = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/successed_images.pkl')

In [56]:
for p in succ_list:
    with open(p,'rb') as f:
        nb_imgs = pickle.load(f)['clean'].shape[0]
        adv_method = os.path.basename(os.path.dirname(p))
        adv_df.loc[adv_method,'# Success Images'] = nb_imgs

adv_df['# Success Images'] = adv_df['# Success Images'].astype(int)

In [57]:
print(adv_df.to_markdown())

|            |   Adv Acc(%) |   Adv Acc(%) DWT |   # Success Images |
|:-----------|-------------:|-----------------:|-------------------:|
| DeepFool   |         4.28 |            78.8  |              23173 |
| BIM        |         2.42 |            31.17 |              23615 |
| CW         |        33.62 |            74.71 |              15495 |
| Low_CW     |        76.69 |            93.47 |               4394 |
| FAB        |         1.24 |            93.41 |              23938 |
| FGSM       |        40.35 |            55.49 |              13760 |
| PGD        |         2.74 |            33.89 |              23566 |
| Low_PGD1   |        81.72 |            92.56 |               3142 |
| Low_PGD2   |        49.66 |            83.99 |              11355 |
| AutoAttack |         0.43 |            38.36 |              24129 |
| Square     |         4.81 |            81.37 |              23008 |


# Detection

## Known Attack

In [58]:
del_keys = ['TNR','AUIN','AUOUT']

known_results = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/result.json')
known_df = pd.DataFrame()

for p in known_results:
    r = json.load(open(p,'r'))['test']

    for k in del_keys:
        del r[k]

    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    
    known_df = pd.concat([known_df, r], axis=1)

known_df = ((known_df.T.loc[order]) * 100).round(2)

known_df.columns = ['AUROC(%)','Detection Acc(%)']

## Train and Test images

In [59]:
train_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/train.pt')
dev_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/dev.pt')
test_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/test.pt')

In [60]:
for train_p, dev_p, test_p in zip(train_det, dev_det, test_det):
    nb_train = torch.load(train_p)['logits'].size(0)
    nb_dev = torch.load(dev_p)['logits'].size(0)    
    nb_test = torch.load(test_p)['logits'].size(0)
    
    adv_method = os.path.basename(os.path.dirname(train_p))
    
    known_df.loc[adv_method, '#(train, dev, test)'] = f"({nb_train}, {nb_dev}, {nb_test})"

In [61]:
print(known_df.to_markdown())

|            |   AUROC(%) |   Detection Acc(%) | #(train, dev, test)   |
|:-----------|-----------:|-------------------:|:----------------------|
| DeepFool   |      94.33 |              88.66 | (16354, 2725, 8180)   |
| BIM        |      99.57 |              97.43 | (23573, 3927, 11790)  |
| CW         |      90.29 |              84.11 | (11686, 1947, 5845)   |
| Low_CW     |      87.5  |              83.36 | (3035, 504, 1523)     |
| FAB        |      96.35 |              91.42 | (15002, 2499, 7504)   |
| FGSM       |      89.32 |              82.5  | (12162, 2026, 6084)   |
| PGD        |      99.49 |              97.21 | (23148, 3857, 11576)  |
| Low_PGD1   |      81.22 |              76.09 | (2268, 377, 1136)     |
| Low_PGD2   |      89.78 |              82.66 | (8342, 1389, 4174)    |
| AutoAttack |      99.54 |              97.3  | (22959, 3825, 11482)  |
| Square     |      98.23 |              95.14 | (15704, 2616, 7855)   |


## Transfer Attack

In [62]:
transfer_df = pd.read_csv(f'./results/{dataname}/transfer_attack_results/{modelname}/transfer_results.csv',index_col=0)
print(transfer_df.loc[order, order].to_markdown())

|            |   DeepFool |   BIM |    CW |   Low_CW |   FAB |   FGSM |   PGD |   Low_PGD1 |   Low_PGD2 |   AutoAttack |   Square |
|:-----------|-----------:|------:|------:|---------:|------:|-------:|------:|-----------:|-----------:|-------------:|---------:|
| DeepFool   |      94.33 | 53.61 | 93.78 |    89.5  | 91.69 |  91.08 | 55.08 |      86.06 |      77.35 |        52.81 |    92.63 |
| BIM        |      34.16 | 99.57 | 46.25 |    23.13 | 24.32 |  55.06 | 99.63 |      55.35 |      94.76 |        99.56 |    15.03 |
| CW         |      88.69 | 58.87 | 90.29 |    86.35 | 86.55 |  88.29 | 61.13 |      84.75 |      79.73 |        59.11 |    86.49 |
| Low_CW     |      87.76 | 54.83 | 87.18 |    87.5  | 87.62 |  80.5  | 57.36 |      87.41 |      80    |        57.78 |    89.15 |
| FAB        |      96.59 | 56.24 | 95.28 |    93.84 | 96.35 |  92.54 | 58.97 |      88.04 |      78.36 |        56.18 |    97.06 |
| FGSM       |      85.22 | 60.89 | 87.28 |    81.43 | 83.11 |  89.32 | 62.8